### Exercises

### Do your work for this exercise in a jupyter notebook named feature_engineering within the regression-exercises repo. Add, commit, and push your work.

### 1. Load the tips dataset.

In [1]:
import pandas as pd
import numpy as np
import wrangle
import warnings
warnings.filterwarnings("ignore")
from pydataset import data
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

from sklearn.model_selection import train_test_split
import sklearn.preprocessing

In [2]:
df = data('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


- a. Create a column named tip_percentage. This should be the tip amount divided by the total bill.

In [3]:
df['tip_percentage'] = round((df.tip/df.total_bill)*100, 2)
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,5.94
2,10.34,1.66,Male,No,Sun,Dinner,3,16.05
3,21.01,3.50,Male,No,Sun,Dinner,3,16.66
4,23.68,3.31,Male,No,Sun,Dinner,2,13.98
5,24.59,3.61,Female,No,Sun,Dinner,4,14.68


In [4]:
df = df.rename(columns = {'size':'num_person'})
df.head()

,total_bill,tip,sex,smoker,day,time,num_person,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,5.94
2,10.34,1.66,Male,No,Sun,Dinner,3,16.05
3,21.01,3.50,Male,No,Sun,Dinner,3,16.66
4,23.68,3.31,Male,No,Sun,Dinner,2,13.98
5,24.59,3.61,Female,No,Sun,Dinner,4,14.68


- b. Create a column named price_per_person. This should be the total bill divided by the party size.

In [5]:
df['price_per_person']= round((df.total_bill/df.num_person), 2)
df.head()

,total_bill,tip,sex,smoker,day,time,num_person,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,5.94,8.49
2,10.34,1.66,Male,No,Sun,Dinner,3,16.05,3.45
3,21.01,3.50,Male,No,Sun,Dinner,3,16.66,7.00
4,23.68,3.31,Male,No,Sun,Dinner,2,13.98,11.84
5,24.59,3.61,Female,No,Sun,Dinner,4,14.68,6.15


In [6]:
df['sex']=df.sex.str.strip()


In [7]:
df['smoker']=df.smoker.str.strip()

In [8]:
df['day']=df.day.str.strip()

In [9]:
df['time']=df.time.str.strip()

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 1 to 244
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   total_bill        244 non-null    float64
 1   tip               244 non-null    float64
 2   sex               244 non-null    object 
 3   smoker            244 non-null    object 
 4   day               244 non-null    object 
 5   time              244 non-null    object 
 6   num_person        244 non-null    int64  
 7   tip_percentage    244 non-null    float64
 8   price_per_person  244 non-null    float64
dtypes: float64(4), int64(1), object(4)
memory usage: 19.1+ KB


In [11]:
# Convert binary categorical variables to numeric
df['sex_encoded'] = df.sex.map({'Female': 1, 'Male': 0})
df['smoker_encoded'] = df.smoker.map({'No': 0, 'Yes': 1})
# Get dummies for non-binary categorical variables
dummy_df = pd.get_dummies(df[['day',
                              'time']], dummy_na=False,
                              drop_first=True)

df.head()

,total_bill,tip,sex,smoker,day,time,num_person,tip_percentage,price_per_person,sex_encoded,smoker_encoded
1,16.99,1.01,Female,No,Sun,Dinner,2,5.94,8.49,1,0
2,10.34,1.66,Male,No,Sun,Dinner,3,16.05,3.45,0,0
3,21.01,3.50,Male,No,Sun,Dinner,3,16.66,7.00,0,0
4,23.68,3.31,Male,No,Sun,Dinner,2,13.98,11.84,0,0
5,24.59,3.61,Female,No,Sun,Dinner,4,14.68,6.15,1,0


In [12]:
# Concatenate dummy dataframe to original
df = pd.concat([df, dummy_df], axis=1)

In [13]:
df.head()

,total_bill,tip,sex,smoker,day,time,num_person,tip_percentage,price_per_person,sex_encoded,smoker_encoded,day_Sat,day_Sun,day_Thur,time_Lunch
1,16.99,1.01,Female,No,Sun,Dinner,2,5.94,8.49,1,0,0,1,0,0
2,10.34,1.66,Male,No,Sun,Dinner,3,16.05,3.45,0,0,0,1,0,0
3,21.01,3.50,Male,No,Sun,Dinner,3,16.66,7.00,0,0,0,1,0,0
4,23.68,3.31,Male,No,Sun,Dinner,2,13.98,11.84,0,0,0,1,0,0
5,24.59,3.61,Female,No,Sun,Dinner,4,14.68,6.15,1,0,0,1,0,0


In [14]:
df = df.drop(columns = ['sex', 'day', 'time', 'smoker'])
df.head(100)


,total_bill,tip,num_person,tip_percentage,price_per_person,sex_encoded,smoker_encoded,day_Sat,day_Sun,day_Thur,time_Lunch
1,16.99,1.01,2,5.94,8.49,1,0,0,1,0,0
2,10.34,1.66,3,16.05,3.45,0,0,0,1,0,0
3,21.01,3.50,3,16.66,7.00,0,0,0,1,0,0
4,23.68,3.31,2,13.98,11.84,0,0,0,1,0,0
5,24.59,3.61,4,14.68,6.15,1,0,0,1,0,0
6,25.29,4.71,4,18.62,6.32,0,0,0,1,0,0
7,8.77,2.00,2,22.81,4.38,0,0,0,1,0,0
8,26.88,3.12,4,11.61,6.72,0,0,0,1,0,0
9,15.04,1.96,2,13.03,7.52,0,0,0,1,0,0
10,14.78,3.23,2,21.85,7.39,0,0,0,1,0,0


In [15]:
df.shape

(244, 11)

- c. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

In [16]:
# I would guess total_bill

In [17]:
# total_bill

In [18]:
def split_tips(df):
    '''
    This function performs split on telco data, stratify churn.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2,
                                            random_state=123,
                                           )
    train, validate = train_test_split(train_validate, test_size=.2,
                                       random_state=123,
                                       )
    return train, validate, test

In [19]:
# Split the data
train, validate, test = split_tips(df)

In [20]:
target = "tip"

# split train into X (dataframe, drop target) & y (series, keep target only)
X_train = train.drop(columns=[target])
y_train = train[target]

# split validate into X (dataframe, drop target) & y (series, keep target only)
X_validate = validate.drop(columns=[target])
y_validate = validate[target]

# split test into X (dataframe, drop target) & y (series, keep target only)
X_test = test.drop(columns=[target])
y_test = test[target]

X_train.head()

,total_bill,num_person,tip_percentage,price_per_person,sex_encoded,smoker_encoded,day_Sat,day_Sun,day_Thur,time_Lunch
126,29.80,6,14.09,4.97,1,0,0,0,1,1
189,18.15,3,19.28,6.05,1,1,0,1,0,0
84,32.68,2,15.30,16.34,0,1,0,0,1,1
241,27.18,2,7.36,13.59,1,1,1,0,0,0
40,31.27,3,15.99,10.42,0,0,1,0,0,0


In [21]:
import sklearn.preprocessing
# Scale (Make the thing)
scaler = sklearn.preprocessing.MinMaxScaler()

# Fit the scaler, (fit the thing)
scaler.fit(X_train)

# Use the scaler to transform train, validate, test (use the thing)
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

In [22]:
# Turn everything into a dataframe
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_validate_scaled = pd.DataFrame(X_validate_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_train.columns)

- d. Use select k best and recursive feature elimination to select the top 2 features for predicting tip amount. What are they?

In [23]:
from sklearn.feature_selection import SelectKBest, f_regression 
# Use numeric features to predict tip_amount
# Use select-K-best and RFE to select the top 2 features

k = 2

# Let's start with Select K Best
# Make the thing
kbest = SelectKBest(f_regression, k=2)

# fit the thing
kbest.fit(X_train, y_train)

# use the thing, 
# get_support() produces an array of booleans, so we can filter out the column names that matter the most
kbest_features = X_train.columns[kbest.get_support()].tolist()

print("KBest's 2 best features are", kbest_features)

KBest's 2 best features are ['total_bill', 'num_person']


In [24]:
from sklearn.feature_selection import SelectKBest, f_regression

# parameters: f_regression stats test, give me  features
f_selector = SelectKBest(f_regression, k=2)

# find the top 8 X's correlated with y
f_selector.fit(X_train_scaled, y_train)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_features = X_train_scaled.iloc[:,feature_mask].columns.tolist()

In [25]:
f_features

['total_bill', 'num_person']

In [26]:
# Now let's do the RFE

# Make the thing(s)
lm = sklearn.linear_model.LinearRegression()
rfe = sklearn.feature_selection.RFE(lm, n_features_to_select=2)

# Fit the thing
rfe.fit(X_train, y_train)

# use the thing
rfe_columns = X_train.columns[rfe.support_].tolist()
rfe_columns

['day_Sat', 'day_Thur']

In [47]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(estimator=lm, n_features_to_select=2)

# fit the data using RFE
rfe.fit(X_train_scaled,y_train)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

In [48]:
rfe_feature

['total_bill', 'tip']

- e. Use select k best and recursive feature elimination to select the top 2 features for predicting tip percentage. What are they?

In [28]:
target = "tip_percentage"

# split train into X (dataframe, drop target) & y (series, keep target only)
X_train = train.drop(columns=[target])
y_train = train[target]

# split validate into X (dataframe, drop target) & y (series, keep target only)
X_validate = validate.drop(columns=[target])
y_validate = validate[target]

# split test into X (dataframe, drop target) & y (series, keep target only)
X_test = test.drop(columns=[target])
y_test = test[target]

X_train.head()

,total_bill,tip,num_person,price_per_person,sex_encoded,smoker_encoded,day_Sat,day_Sun,day_Thur,time_Lunch
126,29.80,4.2,6,4.97,1,0,0,0,1,1
189,18.15,3.5,3,6.05,1,1,0,1,0,0
84,32.68,5.0,2,16.34,0,1,0,0,1,1
241,27.18,2.0,2,13.59,1,1,1,0,0,0
40,31.27,5.0,3,10.42,0,0,1,0,0,0


In [29]:
import sklearn.preprocessing
# Scale (Make the thing)
scaler = sklearn.preprocessing.MinMaxScaler()

# Fit the scaler, (fit the thing)
scaler.fit(X_train)

# Use the scaler to transform train, validate, test (use the thing)
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

In [30]:
# Turn everything into a dataframe
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_validate_scaled = pd.DataFrame(X_validate_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_train.columns)

In [31]:
# Use numeric features to predict tip_percentage
# Use select-K-best and RFE to select the top 2 features

k = 2

# Let's start with Select K Best
# Make the thing
kbest = SelectKBest(f_regression, k=2)

# fit the thing
kbest.fit(X_train, y_train)

# use the thing, 
# get_support() produces an array of booleans, so we can filter out the column names that matter the most
kbest_features_tp = X_train.columns[kbest.get_support()].tolist()

print("KBest's 2 best features are", kbest_features_tp)

KBest's 2 best features are ['total_bill', 'tip']


In [32]:
# Now let's do the RFE

# Make the thing(s)
lm = sklearn.linear_model.LinearRegression()
rfe = sklearn.feature_selection.RFE(lm, n_features_to_select=2)

# Fit the thing
rfe.fit(X_train, y_train)

# use the thing
rfe_columns = X_train.columns[rfe.support_].tolist()
rfe_columns

['tip', 'num_person']

In [33]:
from sklearn.feature_selection import SelectKBest, f_regression

# parameters: f_regression stats test, give me 2 features
f_selector = SelectKBest(f_regression, k=2)

# find the top 2 X's correlated with y
f_selector.fit(X_train, y_train)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature1 = X_train.iloc[:,feature_mask].columns.tolist()

In [34]:
f_feature1

['total_bill', 'tip']

In [35]:
from sklearn.feature_selection import SelectKBest, f_regression

# parameters: f_regression stats test, give me 3 features
f_selector = SelectKBest(f_regression, k=3)

# find the top 8 X's correlated with y
f_selector.fit(X_train_scaled, y_train)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

In [36]:
f_feature

['total_bill', 'tip', 'price_per_person']

In [37]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
# rfe = RFE(lm, 2)
rfe = RFE(estimator=lm, n_features_to_select=2) 

# fit the data using RFE
rfe.fit(X_train_scaled,y_train)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

In [38]:
rfe_feature

['total_bill', 'tip']

- f. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

In [ ]:
# I don't know why is it so.

### 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [39]:
def select_kbest(X, y, k):
    # make the object
    kbest = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_regression, k=k)

    # fit the object
    kbest.fit(X, y)
    
    # use the object (.get_support() is that array of booleans to filter the list of column names)
    return X.columns[kbest.get_support()].tolist()

select_kbest(X_train, y_train, 2)

['total_bill', 'tip']

In [40]:
def show_features_rankings(X_train, rfe):
    """
    Takes in a dataframe and a fit RFE object in order to output the rank of all features
    """
    # rfe here is reference rfe from cell 15
    var_ranks = rfe.ranking_
    var_names = X_train.columns.tolist()
    ranks = pd.DataFrame({'Var': var_names, 'Rank': var_ranks})
    ranks = ranks.sort_values(by="Rank", ascending=True)
    return ranks

### 3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [41]:
# Exercise 3
# Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

def select_rfe(X, y, k):
    # make the thing
    lm = sklearn.linear_model.LinearRegression()
    rfe = sklearn.feature_selection.RFE(lm, n_features_to_select=k)

    # Fit the thing
    rfe.fit(X, y)
    
    # use the thing
    features_to_use = X.columns[rfe.support_].tolist()
    
    # we need to send show_feature_rankings a trained/fit RFE object
    all_rankings = show_features_rankings(X, rfe)
    
    return features_to_use, all_rankings
select_rfe(X_train, y_train, 2)

(['tip', 'num_person'],
                 Var  Rank
 1               tip     1
 2        num_person     1
 5    smoker_encoded     2
 3  price_per_person     3
 7           day_Sun     4
 8          day_Thur     5
 6           day_Sat     6
 0        total_bill     7
 4       sex_encoded     8
 9        time_Lunch     9)

### 4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [49]:
swiss_df = data('swiss')
swiss_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, Courtelary to Rive Gauche
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fertility         47 non-null     float64
 1   Agriculture       47 non-null     float64
 2   Examination       47 non-null     int64  
 3   Education         47 non-null     int64  
 4   Catholic          47 non-null     float64
 5   Infant.Mortality  47 non-null     float64
dtypes: float64(4), int64(2)
memory usage: 2.6+ KB


In [50]:
swiss_df.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [52]:
swiss_df.describe()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
count,47.000000,47.000000,47.000000,47.000000,47.00000,47.000000
mean,70.142553,50.659574,16.489362,10.978723,41.14383,19.942553
std,12.491697,22.711218,7.977883,9.615407,41.70485,2.912697
min,35.000000,1.200000,3.000000,1.000000,2.15000,10.800000
25%,64.700000,35.900000,12.000000,6.000000,5.19500,18.150000
50%,70.400000,54.100000,16.000000,8.000000,15.14000,20.000000
75%,78.450000,67.650000,22.000000,12.000000,93.12500,21.700000
max,92.500000,89.700000,37.000000,53.000000,100.00000,26.600000


In [53]:
swiss_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, Courtelary to Rive Gauche
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fertility         47 non-null     float64
 1   Agriculture       47 non-null     float64
 2   Examination       47 non-null     int64  
 3   Education         47 non-null     int64  
 4   Catholic          47 non-null     float64
 5   Infant.Mortality  47 non-null     float64
dtypes: float64(4), int64(2)
memory usage: 2.6+ KB


In [54]:
# Split the data
train, validate, test = split_tips(swiss_df)

In [55]:
print(train.shape)
print(validate.shape)
print(test.shape)

(29, 6)
(8, 6)
(10, 6)


In [56]:
target = "Fertility"

# split train into X (dataframe, drop target) & y (series, keep target only)
X_train = train.drop(columns=["Fertility"])
y_train = train["Fertility"]

# split validate into X (dataframe, drop target) & y (series, keep target only)
X_validate = validate.drop(columns=["Fertility"])
y_validate = validate["Fertility"]

# split test into X (dataframe, drop target) & y (series, keep target only)
X_test = test.drop(columns=["Fertility"])
y_test = test["Fertility"]

X_train.head()

,Agriculture,Examination,Education,Catholic,Infant.Mortality
Rive Gauche,27.7,22,29,58.33,19.3
Courtelary,17.0,15,12,9.96,22.2
Rive Droite,46.6,16,29,50.43,18.2
Aubonne,67.5,14,7,2.27,19.1
Rolle,60.8,16,10,7.72,16.3


In [57]:
import sklearn.preprocessing
# Scale (Make the thing)
scaler = sklearn.preprocessing.MinMaxScaler()

# Fit the scaler, (fit the thing)
scaler.fit(X_train)

# Use the scaler to transform train, validate, test (use the thing)
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

In [58]:
# Turn everything into a dataframe
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_validate_scaled = pd.DataFrame(X_validate_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_train.columns)

In [59]:
# Using functions defined in # 2 to get kbest features
select_kbest(X_train_scaled, y_train, 3)

['Examination', 'Education', 'Infant.Mortality']

In [60]:
# Using functions defined in # 3 to get rfe featurea and rankings
select_rfe(X_train_scaled, y_train, 3)

(['Examination', 'Education', 'Infant.Mortality'],
                 Var  Rank
 1       Examination     1
 2         Education     1
 4  Infant.Mortality     1
 0       Agriculture     2
 3          Catholic     3)